In [1]:
import os
import pickle
import re
import time

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from urllib.parse import urlencode


### Collection FEI Show Jumping Data

Example web scraping at cs109a_hw1_solutions.ipynb

First goal: Get data from one speed competition across multiple pages.

In [2]:
# In this notebook, pass this as the `headers` argument when using requests.get()
headers = {'User-Agent': 'Mozilla/5.0'}
snapshot_url = 'https://data.fei.org/Result/ResultList.aspx?p=9A80682A95C06DB56193A83C95F324D39683772B3B2F77A7BC2A1E3E667ABB0A85CA703CF95D562D86A621768D9E7146' # SOLUTION
snapshot_response = requests.get(snapshot_url, headers=headers) # SOLUTION
status_code = snapshot_response.status_code  # SOLUTION
print(f"Response statuse code: {status_code}")

Response statuse code: 403


In [3]:
# response = requests.get(snapshot_url, params={'page': 0})

# page2_soup = BeautifulSoup(response.content, "html.parser")
# print(page2_soup.prettify())

In [4]:
proxy_params = {
      'api_key': '2b680f1f-2808-48e3-9059-c1615c97f189',
      'url': 'https://data.fei.org/Result/ResultList.aspx?p=9A80682A95C06DB56193A83C95F324D39683772B3B2F77A7BC2A1E3E667ABB0A85CA703CF95D562D86A621768D9E7146', 
  }

speed_response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params=urlencode(proxy_params),
  timeout=120,
)

print('Body: ', speed_response.content)

Body:  b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n\r\n<head id="Head1">\r\n\r\n\t<!--meta http-equiv="Cache-Control" content="No-Cache" /-->\r\n\t<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />\r\n\t\t<title>\r\n\t\t\t\r\n\tCompetition Results\r\n\r\n\t\t</title>\r\n        <!-- Google Tag Manager -->\r\n                <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\n                new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\n                j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n                \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n                })(window,document,\'script\',\'dataLayer\',\'GTM-TTFL8BB\');</script>\r\n                <!-- End Google Tag Manager -->\r\n        <sc

In [5]:
speed_soup = BeautifulSoup(speed_response.content, "html.parser")

# Taste the soup! You should see HTML code here
print(speed_soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head id="Head1">
  <!--meta http-equiv="Cache-Control" content="No-Cache" /-->
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Competition Results
  </title>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
                new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
                j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
                'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
                })(window,document,'script','dataLayer','GTM-TTFL8BB');
  </script>
  <!-- End Google Tag Manager -->
  <script src="/Scripts/webforms?v=pKnet6fpKWejMp3wsi6ocB4W5e7J6diif1Ok6ZLaL5A1">
  </script>
  <script src="/_layouts/FEI/webforms?v=teiyTm0pq

In [6]:
# # Get page 2 data
# session = requests.Session()
# response = session.get(snapshot_url)

# viewstate = speed_soup.find("input", {"name": "__VIEWSTATE"}).get("value")
# eventvalidation = speed_soup.find("input", {"name": "__EVENTVALIDATION"}).get("value")

# # Prepare payload to request the second page
# payload = {
#     "__EVENTTARGET": "ctl00$PlaceHolderMain$gvcIResults",
#     "__EVENTARGUMENT": "Page$2",
#     "__VIEWSTATE": viewstate,
#     "__EVENTVALIDATION": eventvalidation,
# }

# # Send the request to get the second page
# response_page_2 = session.post(snapshot_url, data=payload)
# soup_page_2 = BeautifulSoup(response_page_2.text, 'html.parser')

# print(soup_page_2.prettify())


In [7]:
# Provided code
def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_from_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)



In [8]:
# Extract competition ID

def extract_ci_string(text):
    text = text.decode('utf-8')
    # Regex to capture any part of the string containing '_CI_'
    pattern = r'\b\w+_CI_\w+\b'
    matches = re.findall(pattern, text)
    
    if len(matches) == 1:
        return matches[0]  # Return the match if there's exactly one
    elif len(matches) > 1:
        raise ValueError("Error: More than one '_CI_' string found.")
    else:
        raise ValueError("Error: No '_CI_' string found.")
    
CI_result = extract_ci_string(speed_response.content)
print("Extracted String:", CI_result)

Extracted String: 2023_CI_0657_S_U25_01_01


## Data Table for Speed Competition
The following function takes in Beautiful Soup object and outputs a list of dictionaries. 

Each dictionary holds the following values: 
{
    'competition_id' : '',
    'position' : '',
    'athlete_fei_id' : '',
    'athlete' : '',
    'horse_fei_id' : '',
    'horse' : '',
    'phase_number' : '',
    'time_faults' : '',
    'jumping_faults' : '', 
    'time' : ''
}

In [9]:
def parse_speed_page(text, filename="speed_data.csv"):
    speed_soup = BeautifulSoup(text, "html.parser")
    competition_id = extract_ci_string(text)
    
    print("comp id:", competition_id)
    table = speed_soup.find('table', {'id': 'PlaceHolderMain_gvcIResults'})

    # Extract the table rows
    rows = table.find_all('tr')

    # Process table headers
    headers = [header.text.strip() for header in rows[2].find_all('th')]

    rows = table.find_all('tr', class_=['row', 'altrow'])

    data = []
    for index, row in enumerate(rows): 
        if index == 16:
            break 
        #print(f"row {index}: {row}")
        cols = row.find_all('td')
        if len(cols) < 9:
            continue
        
        # Extract the required data
        competition_data = {
            'competition_id': competition_id,
            'position': cols[0].get_text(strip=True),
            'athlete_fei_id': cols[1].find('a').get_text(strip=True),
            'athlete': cols[2].find('a').get_text(strip=True),
            'horse_fei_id': cols[3].find('a').get_text(strip=True),
            'horse': cols[4].find('a').get_text(strip=True),
            'phase_number': '1',  # Assuming 'Round 1' or similar context, adjust if necessary
            'total_faults': cols[8].find_all('td')[1].get_text(strip=True),
            'time': cols[8].find_all('td')[3].get_text(strip=True),
        }
        
        data.append(competition_data)

    #save_to_pickle(data, filename)
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

    return df
    


speed_data = parse_speed_page(speed_response.content, 'u25_speed_data.csv')
speed_data

comp id: 2023_CI_0657_S_U25_01_01


,competition_id,position,athlete_fei_id,athlete,horse_fei_id,horse,phase_number,total_faults,time
0,2023_CI_0657_S_U25_01_01,1,10305300,Luke JENSEN (USA),108EG60,FAYURI,1,0,56.09
1,2023_CI_0657_S_U25_01_01,2,10071490,Zoe CONTER (BEL),105MI69,LAZY,1,0,59.00
2,2023_CI_0657_S_U25_01_01,3,10237647,Sofia CADY (USA),106RW73,ZG BELLA,1,0,61.80
3,2023_CI_0657_S_U25_01_01,4,10148821,Ansgar HOLTGERS JR (USA),104SJ54,CHANYON,1,0,63.63
4,2023_CI_0657_S_U25_01_01,5,10146449,Natalie DEAN (USA),106QD02,OASIS VAN DE BUCXTALE,1,0,63.76
5,2023_CI_0657_S_U25_01_01,6,10227949,Annie BOLLING (USA),107DZ85,MYKONOS SCF,1,0,65.03
6,2023_CI_0657_S_U25_01_01,7,10138651,Mimi GOCHMAN (USA),106EX70,COSMOS BH,1,0,65.28
7,2023_CI_0657_S_U25_01_01,8,10109611,Karime PEREZ NUÑEZ (MEX),106CA41,CAMINA K,1,0,65.63
8,2023_CI_0657_S_U25_01_01,9,10151449,Eva FISHERMAN (USA),105JT83,CASALLVANO,1,0,65.83
9,2023_CI_0657_S_U25_01_01,10,10206787,Elena A. HAAS (USA),105CD00,CENTANA,1,0,66.20


In [10]:
table = speed_soup.find('table', {'id': 'PlaceHolderMain_gvcIResults'})
print(table)

<table cellpadding="0" cellspacing="0" class="grid sc resgrid" id="PlaceHolderMain_gvcIResults" style="border-collapse:collapse;">
<tr class="pager">
<td colspan="9"><div id="PlaceHolderMain_gvcIResults_pnlPager">
<table>
<tr>
<td style="width:100%;white-space:nowrap;">64 Result(s)  /  2 Page(s)</td><td style="width:100%;"></td><td>Page</td><td><input alt="Previous" name="ctl00$PlaceHolderMain$gvcIResults$ctl01$ctl01" src="../Style%20Library/Images/btn-back.gif" type="image"/></td><td><span>1</span></td><td><a href="javascript:__doPostBack('ctl00$PlaceHolderMain$gvcIResults','Page$2')">2</a></td><td><input alt="Next" name="ctl00$PlaceHolderMain$gvcIResults$ctl01$ctl04" src="../Style%20Library/Images/btn-forward.gif" type="image"/></td>
</tr>
</table>
</div></td>
</tr><tr align="left" class="header">
<th align="right" class="headerrsort" scope="col" style="width:3%;"><a href="javascript:__doPostBack('ctl00$PlaceHolderMain$gvcIResults','Sort$position')">Pos.</a></th><th class="header" sc